In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

import os
import cv2

In [2]:
data = pd.read_csv("fer2013new.csv")

In [3]:
data.head()

,Usage,Image name,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,NF
0,Training,fer0000000.png,4,0,0,1,3,2,0,0,0,0
1,Training,fer0000001.png,6,0,1,1,0,0,0,0,2,0
2,Training,fer0000002.png,5,0,0,3,1,0,0,0,1,0
3,Training,fer0000003.png,4,0,0,4,1,0,0,0,1,0
4,Training,fer0000004.png,9,0,0,1,0,0,0,0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Usage       35887 non-null  object
 1   Image name  35714 non-null  object
 2   neutral     35887 non-null  int64 
 3   happiness   35887 non-null  int64 
 4   surprise    35887 non-null  int64 
 5   sadness     35887 non-null  int64 
 6   anger       35887 non-null  int64 
 7   disgust     35887 non-null  int64 
 8   fear        35887 non-null  int64 
 9   contempt    35887 non-null  int64 
 10  unknown     35887 non-null  int64 
 11  NF          35887 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 3.3+ MB


In [5]:
data.describe()

,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,NF
count,35887.000000,35887.000000,35887.000000,35887.000000,35887.000000,35887.000000,35887.000000,35887.000000,35887.000000,35887.000000
mean,2.995096,2.548165,1.156129,1.322094,0.793268,0.150222,0.339231,0.168919,0.472706,0.054142
std,3.342949,3.949983,2.499870,2.326627,1.964314,0.589273,1.075407,0.618953,0.805554,0.702497
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.000000,5.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,8.000000,10.000000


### Очистить от несуществующих значений

In [6]:
data = data.dropna()

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35714 entries, 0 to 35886
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Usage       35714 non-null  object
 1   Image name  35714 non-null  object
 2   neutral     35714 non-null  int64 
 3   happiness   35714 non-null  int64 
 4   surprise    35714 non-null  int64 
 5   sadness     35714 non-null  int64 
 6   anger       35714 non-null  int64 
 7   disgust     35714 non-null  int64 
 8   fear        35714 non-null  int64 
 9   contempt    35714 non-null  int64 
 10  unknown     35714 non-null  int64 
 11  NF          35714 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 3.5+ MB


In [8]:
data.describe()

,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt,unknown,NF
count,35714.000000,35714.000000,35714.000000,35714.000000,35714.000000,35714.000000,35714.000000,35714.000000,35714.000000,35714.000000
mean,3.009604,2.560508,1.161729,1.328499,0.797110,0.150949,0.340875,0.169737,0.474996,0.005964
std,3.344515,3.955545,2.504619,2.330431,1.968288,0.590606,1.077749,0.620338,0.806829,0.119937
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6.000000,5.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,8.000000,10.000000


### Удалить строчки где столбец NF равен 10 (значит картинка не была найдена опрошенными) 

In [9]:
data = data[data.NF != 10]

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35711 entries, 0 to 35886
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Usage       35711 non-null  object
 1   Image name  35711 non-null  object
 2   neutral     35711 non-null  int64 
 3   happiness   35711 non-null  int64 
 4   surprise    35711 non-null  int64 
 5   sadness     35711 non-null  int64 
 6   anger       35711 non-null  int64 
 7   disgust     35711 non-null  int64 
 8   fear        35711 non-null  int64 
 9   contempt    35711 non-null  int64 
 10  unknown     35711 non-null  int64 
 11  NF          35711 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 3.5+ MB


### Удалить столбец unknown (Этот столбец был отмечен опрошенными если они не могли понять что это за эмоция) и столбец NF

In [11]:
data = data.drop(columns=["NF", "unknown"])

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35711 entries, 0 to 35886
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Usage       35711 non-null  object
 1   Image name  35711 non-null  object
 2   neutral     35711 non-null  int64 
 3   happiness   35711 non-null  int64 
 4   surprise    35711 non-null  int64 
 5   sadness     35711 non-null  int64 
 6   anger       35711 non-null  int64 
 7   disgust     35711 non-null  int64 
 8   fear        35711 non-null  int64 
 9   contempt    35711 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 3.0+ MB


### Преобразовать голоса в вероятности

In [13]:
converted_data = data.copy()
for col in converted_data[["neutral", "happiness", "surprise", "sadness", "anger", "disgust", "fear", "contempt"]]:
    converted_data[col] = data[col] / data[["neutral", "happiness", "surprise", "sadness", "anger", "disgust", "fear", "contempt"]].sum(axis=1)
data = converted_data
data.head()

,Usage,Image name,neutral,happiness,surprise,sadness,anger,disgust,fear,contempt
0,Training,fer0000000.png,0.400000,0.0,0.000,0.100000,0.300000,0.2,0.0,0.0
1,Training,fer0000001.png,0.750000,0.0,0.125,0.125000,0.000000,0.0,0.0,0.0
2,Training,fer0000002.png,0.555556,0.0,0.000,0.333333,0.111111,0.0,0.0,0.0
3,Training,fer0000003.png,0.444444,0.0,0.000,0.444444,0.111111,0.0,0.0,0.0
4,Training,fer0000004.png,0.900000,0.0,0.000,0.100000,0.000000,0.0,0.0,0.0


In [14]:
data = data.replace("Training", "train")
data = data.replace("PublicTest", "validation")
data = data.replace("PrivateTest", "test")
data.Usage.value_counts()

train         28559
validation     3579
test           3573
Name: Usage, dtype: int64

### Проверить существуют ли файлы в папке

In [15]:
#Check if all files are in place
train_path = "./fer2013/train"
test_path = "./fer2013/test"
train = os.listdir(train_path)
test = os.listdir(test_path)
dirs = train + test
data = data.loc[data["Image name"].isin(dirs)]
data.Usage.value_counts()

train         28389
validation     3553
test           3546
Name: Usage, dtype: int64

In [16]:
data.to_csv("ferplus_filtered.csv")

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35488 entries, 0 to 35886
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Usage       35488 non-null  object 
 1   Image name  35488 non-null  object 
 2   neutral     35488 non-null  float64
 3   happiness   35488 non-null  float64
 4   surprise    35488 non-null  float64
 5   sadness     35488 non-null  float64
 6   anger       35488 non-null  float64
 7   disgust     35488 non-null  float64
 8   fear        35488 non-null  float64
 9   contempt    35488 non-null  float64
dtypes: float64(8), object(2)
memory usage: 3.0+ MB
